<a href="https://colab.research.google.com/github/shivamsingh163248/ML_AII_LAB/blob/main/AII/LAB_3_Informed_Search_Techniques_(Heuristic_Based_Search).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔹 PART 1: Theory

What is Informed Search?

    Informed search uses extra knowledge (heuristics) to make searching faster and smarter.

    It estimates how far a state is from the goal.

    Example algorithms: Greedy Best-First Search, A* (A-star) Search.

What is a Heuristic?

    A heuristic is a function h(n) that estimates the cost from a node n to the goal.

    Example: "Manhattan Distance" in grids.

🔹 PART 2: Problems: Vacuum World and Maze Problem
🧹 VACUUM WORLD PROBLEM (Informed Search)

Problem Description:

    A robot can move left or right and clean.

    Goal: Clean all dirty tiles.

# 1. Vacuum World using A* Search (with heuristics)

In [1]:
import heapq

class VacuumWorld:
    def __init__(self, state, cost=0):
        self.state = state  # [left_tile, right_tile, robot_position]
        self.cost = cost

    def is_goal(self):
        return self.state[0] == 'clean' and self.state[1] == 'clean'

    def successors(self):
        actions = []
        left, right, pos = self.state

        # Move left
        if pos == 'right':
            actions.append(VacuumWorld([left, right, 'left'], self.cost + 1))

        # Move right
        if pos == 'left':
            actions.append(VacuumWorld([left, right, 'right'], self.cost + 1))

        # Clean current tile
        if pos == 'left' and left == 'dirty':
            actions.append(VacuumWorld(['clean', right, pos], self.cost + 1))
        if pos == 'right' and right == 'dirty':
            actions.append(VacuumWorld([left, 'clean', pos], self.cost + 1))

        return actions

    def heuristic(self):
        left, right, _ = self.state
        return (left == 'dirty') + (right == 'dirty')

    def __lt__(self, other):
        return (self.cost + self.heuristic()) < (other.cost + other.heuristic())

def a_star_vacuum(start_state):
    open_list = []
    heapq.heappush(open_list, start_state)
    visited = set()

    while open_list:
        current = heapq.heappop(open_list)

        if tuple(current.state) in visited:
            continue

        visited.add(tuple(current.state))

        if current.is_goal():
            return current

        for successor in current.successors():
            heapq.heappush(open_list, successor)

    return None

# Initial State: Left = dirty, Right = dirty, Robot on left
start = VacuumWorld(['dirty', 'dirty', 'left'])

result = a_star_vacuum(start)
if result:
    print("Reached goal!")
    print("Total cost (steps):", result.cost)
else:
    print("No solution found.")


Reached goal!
Total cost (steps): 3


# 🧩 MAZE PROBLEM (Informed Search)

Problem Description:

    Find the shortest path from start to goal in a maze/grid.

    Heuristic: Manhattan distance (rows + columns difference).

## 2. Maze Solver using A* Search

In [2]:
import heapq

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar_maze(maze, start, goal):
    rows, cols = len(maze), len(maze[0])
    open_list = [(0 + manhattan_distance(start, goal), 0, start, [])]  # (f, g, position, path)
    visited = set()

    while open_list:
        f, g, current, path = heapq.heappop(open_list)

        if current == goal:
            return path + [current]

        if current in visited:
            continue

        visited.add(current)

        x, y = current
        neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]  # up, down, left, right

        for nx, ny in neighbors:
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] != 1:  # 1 = wall
                new_path = path + [current]
                heapq.heappush(open_list, (g + 1 + manhattan_distance((nx, ny), goal), g + 1, (nx, ny), new_path))

    return None

# 0 = Free path, 1 = Wall
maze = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0]
]

start = (0, 0)
goal = (3, 4)

path = astar_maze(maze, start, goal)
print("Path found:", path)


Path found: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (3, 4)]
